# Sales Prediction Model Building Project

## Project Overview
This project aims to build a machine learning model to predict sales based on various factors including store characteristics, temporal features, promotions, and customer traffic.

### Dataset Information
- **Target Variable**: `sales` - Daily sales amount
- **Features**: Store ID, day of week, date, customer count, store status, promotions, holidays
- **Dataset Size**: ~640k records
- **Task Type**: Regression (predicting continuous sales values)

## 1. Project Setup and Data Loading

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")

In [ ]:
# Load the dataset
df = pd.read_csv('../data/sales.csv', index_col=0)

print(f"Dataset shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\nFirst few rows:")
df.head()

## 2. Exploratory Data Analysis (EDA)

In [ ]:
# Basic dataset information
print("=== DATASET OVERVIEW ===")
print(f"Shape: {df.shape}")
print(f"\nData types:")
print(df.dtypes)
print(f"\nBasic statistics:")
df.describe()

In [ ]:
# Check for missing values
print("=== MISSING VALUES ===")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percent
})
print(missing_df[missing_df['Missing Count'] > 0])

In [ ]:
# Target variable analysis
print("=== TARGET VARIABLE ANALYSIS ===")
print(f"Sales statistics:")
print(f"Mean: ${df['sales'].mean():.2f}")
print(f"Median: ${df['sales'].median():.2f}")
print(f"Min: ${df['sales'].min():.2f}")
print(f"Max: ${df['sales'].max():.2f}")
print(f"Standard Deviation: ${df['sales'].std():.2f}")

# Plot sales distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(df['sales'], bins=50, alpha=0.7, edgecolor='black')
axes[0].set_title('Sales Distribution')
axes[0].set_xlabel('Sales Amount')
axes[0].set_ylabel('Frequency')

# Box plot
axes[1].boxplot(df['sales'])
axes[1].set_title('Sales Box Plot')
axes[1].set_ylabel('Sales Amount')

plt.tight_layout()
plt.show()

In [ ]:
# Categorical variables analysis
print("=== CATEGORICAL VARIABLES ANALYSIS ===")
categorical_cols = ['day_of_week', 'open', 'promotion', 'state_holiday', 'school_holiday']

for col in categorical_cols:
    print(f"\n{col.upper()} Value Counts:")
    print(df[col].value_counts().sort_index())

# Visualize categorical variables vs sales
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for i, col in enumerate(categorical_cols):
    if i < len(axes):
        df.groupby(col)['sales'].mean().plot(kind='bar', ax=axes[i])
        axes[i].set_title(f'Average Sales by {col}')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Average Sales')
        axes[i].tick_params(axis='x', rotation=45)

# Remove empty subplot
if len(categorical_cols) < len(axes):
    fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()

## 3. Data Preprocessing and Feature Engineering

In [ ]:
# Create a copy for preprocessing
df_processed = df.copy()

print("=== DATA PREPROCESSING ===")

# Convert date column to datetime
df_processed['date'] = pd.to_datetime(df_processed['date'])

# Extract temporal features
df_processed['year'] = df_processed['date'].dt.year
df_processed['month'] = df_processed['date'].dt.month
df_processed['quarter'] = df_processed['date'].dt.quarter
df_processed['week_of_year'] = df_processed['date'].dt.isocalendar().week

print("Temporal features extracted successfully!")

# Check for any remaining preprocessing needs
print(f"\nProcessed dataset shape: {df_processed.shape}")
print(f"New columns: {list(set(df_processed.columns) - set(df.columns))}")

In [ ]:
# Handle outliers and data quality issues
print("=== OUTLIER DETECTION ===")

# Check for outliers in sales (using IQR method)
Q1 = df_processed['sales'].quantile(0.25)
Q3 = df_processed['sales'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df_processed[(df_processed['sales'] < lower_bound) | (df_processed['sales'] > upper_bound)]
print(f"Number of outliers detected: {len(outliers)}")
print(f"Percentage of outliers: {len(outliers)/len(df_processed)*100:.2f}%")

# Handle closed stores (sales = 0 when open = 0)
closed_stores = df_processed[df_processed['open'] == 0]
print(f"\nClosed store records: {len(closed_stores)}")
print(f"Sales = 0 in closed stores: {(closed_stores['sales'] == 0).sum()}")

# TODO: Decide on outlier treatment strategy (remove, cap, or keep)
print("\nOutlier treatment strategy to be decided based on business context")

## 4. Feature Selection and Correlation Analysis

In [ ]:
# Correlation analysis
print("=== CORRELATION ANALYSIS ===")

# Select numeric columns for correlation
numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
correlation_matrix = df_processed[numeric_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={'shrink': 0.8})
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

# Show correlations with target variable
target_correlations = correlation_matrix['sales'].abs().sort_values(ascending=False)
print("\nCorrelations with Sales (absolute values):")
print(target_correlations)

## 5. Model Preparation and Training

In [ ]:
# Import machine learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline

print("Machine learning libraries imported successfully!")

In [ ]:
# Prepare features and target
print("=== MODEL PREPARATION ===")

# Define feature columns (exclude date and target)
feature_cols = [col for col in df_processed.columns if col not in ['date', 'sales']]
X = df_processed[feature_cols]
y = df_processed['sales']

print(f"Feature columns: {feature_cols}")
print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=None
)

print(f"\nTraining set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
print(f"Train/Test split: {X_train.shape[0]/X.shape[0]:.1%}/{X_test.shape[0]/X.shape[0]:.1%}")

In [ ]:
# Baseline models comparison
print("=== BASELINE MODELS ===")

models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(random_state=42),
    'Lasso Regression': Lasso(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42, n_estimators=100),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42, n_estimators=100)
}

# Store results
results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2
    }
    
    print(f"{name} - RMSE: {rmse:.2f}, MAE: {mae:.2f}, R²: {r2:.4f}")

print("\n=== MODEL COMPARISON ===")
results_df = pd.DataFrame(results).T
print(results_df)

## 6. Model Optimization and Hyperparameter Tuning

In [ ]:
# Hyperparameter tuning for best performing model
print("=== HYPERPARAMETER TUNING ===")

# Choose best model based on R² score
best_model_name = results_df['R²'].idxmax()
print(f"Best performing baseline model: {best_model_name}")

# Example: Hyperparameter tuning for Random Forest
if best_model_name == 'Random Forest':
    print("Tuning Random Forest hyperparameters...")
    
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    # Note: Use a smaller subset for faster tuning in practice
    rf_grid = GridSearchCV(
        RandomForestRegressor(random_state=42),
        param_grid,
        cv=3,  # 3-fold CV for faster execution
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        verbose=1
    )
    
    # Uncomment to run grid search (may take time)
    # rf_grid.fit(X_train, y_train)
    # print(f"Best parameters: {rf_grid.best_params_}")
    # print(f"Best CV score: {-rf_grid.best_score_:.2f}")
    
print("Hyperparameter tuning setup complete!")

## 7. Model Evaluation and Validation

In [ ]:
# Final model evaluation
print("=== FINAL MODEL EVALUATION ===")

# Train final model (use best model from comparison)
final_model = models[best_model_name]
final_model.fit(X_train, y_train)

# Predictions
y_pred_train = final_model.predict(X_train)
y_pred_test = final_model.predict(X_test)

# Comprehensive evaluation
print(f"Final Model: {best_model_name}")
print("\nTraining Set Performance:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")
print(f"MAE: {mean_absolute_error(y_train, y_pred_train):.2f}")
print(f"R²: {r2_score(y_train, y_pred_train):.4f}")

print("\nTest Set Performance:")
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_mae = mean_absolute_error(y_test, y_pred_test)
test_r2 = r2_score(y_test, y_pred_test)
print(f"RMSE: {test_rmse:.2f}")
print(f"MAE: {test_mae:.2f}")
print(f"R²: {test_r2:.4f}")

# Residual analysis
residuals = y_test - y_pred_test

plt.figure(figsize=(15, 5))

# Residuals vs Predictions
plt.subplot(1, 3, 1)
plt.scatter(y_pred_test, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Sales')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted')

# QQ plot of residuals
from scipy import stats
plt.subplot(1, 3, 2)
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals')

# Actual vs Predicted
plt.subplot(1, 3, 3)
plt.scatter(y_test, y_pred_test, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Sales')
plt.ylabel('Predicted Sales')
plt.title('Actual vs Predicted Sales')

plt.tight_layout()
plt.show()